# Replicate the Solar Irradiance Forcing File

In [16]:
import xarray as xr
import cf_xarray

## File trying to recreate

In [45]:
infile = '/glade/p/acom/acom-climate/cesm2/inputdata/atm/cam/solar/SolarForcingNRLSSI2_daily_s18820101_e20171231_c180702.nc'
forcing = xr.open_dataset(infile)

In [46]:
forcing

<xarray.Dataset>
Dimensions:     (wavelength: 3785, time: 49673, bounds: 2)
Coordinates:
  * time        (time) datetime64[ns] 1882-01-01T12:00:00 ... 2017-12-31T12:0...
  * wavelength  (wavelength) float32 115.5 116.5 117.5 ... 9.992e+04 9.998e+04
Dimensions without coordinates: bounds
Data variables:
    band_width  (wavelength) float32 ...
    date        (time) int32 ...
    datesec     (time) int32 ...
    ssi         (time, wavelength) float32 ...
    time_bnds   (time, bounds) datetime64[ns] ...
    tsi         (time) float32 ...
    tsi_unc     (time) float32 ...
Attributes: (12/40)
    Conventions:               CF-1.6, ACDD-1.3
    title:                     Daily SSI calculated using NRL2 solar irradian...
    source:                    NRLSSI2_v02r01
    institution:               Naval Research Laboratory Space Science Divisi...
    standard_name_vocabulary:  CF Standard Name Table v29
    naming_authority:          gov.noaa.ncei
    ...                        ...
    data_description_url:      http://lasp.colorado.edu/lisird/data/nrl2_files
    data_reference:            Coddington, O., J. Lean, P. Pilewskie, M. Snow...
    data_summary:              Daily SSI calculated using NRL2 solar irradian...
    data_title:                NRLSSI2 daily solar irradiance for 18820101-20...
    history:                   Mon Jul  2 18:20:01 2018: ncks -x -v SSI_UNC /...
    NCO:                       netCDF Operators version 4.7.4 (http://nco.sf....

In [17]:
def center_time(ds):
    """make time the center of the time bounds"""
    ds = ds.copy()
    attrs = ds.time.attrs
    encoding = ds.time.encoding
    
    try:
        tb_name, tb_dim = _get_tb_name_and_tb_dim(ds)
        ds['time'] = ds[tb_name].compute().mean(tb_dim).squeeze()
        attrs['note'] = f'time recomputed as {tb_name}.mean({tb_dim})'
        
    except AssertionError:
        print('Using default time values')
    
    ds.time.attrs = attrs
    ds.time.encoding = encoding
    return ds

def _get_tb_name_and_tb_dim(ds):
    """return the name of the time 'bounds' variable and its second dimension"""
    assert 'bounds' in ds.time.attrs, 'missing "bounds" attr on time'
    tb_name = ds.time.attrs['bounds']        
    assert tb_name in ds, f'missing "{tb_name}"'    
    tb_dim = ds[tb_name].dims[-1]
    return tb_name, tb_dim

In [47]:
infile = '../data/ssi_v02r01_daily_s18820101_e20201231_c20210218.nc'
ds = xr.open_dataset(infile)

## Rename variables

* SSI --> ssi
* TSI --> tsi
* TSI_UNC --> tsi_unc
* Wavelength_Band_Width --> band_width

In [48]:
variables_to_rename = {'SSI':'ssi',
                       'TSI':'tsi',
                       'TSI_UNC':'tsi_unc',
                       'Wavelength_Band_Width':'band_width'}

ds = ds.rename(variables_to_rename)

## Add a date value

In [69]:
import pandas as pd
date = pd.to_datetime(ds.time).strftime('%Y%m%d')

In [73]:
ds['date'] = date.astype(int)

In [74]:
ds.time

<xarray.DataArray 'time' (time: 50769)>
array(['1882-01-01T00:00:00.000000000', '1882-01-02T00:00:00.000000000',
       '1882-01-03T00:00:00.000000000', ..., '2020-12-29T00:00:00.000000000',
       '2020-12-30T00:00:00.000000000', '2020-12-31T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1882-01-01 1882-01-02 ... 2020-12-31
Attributes:
    standard_name:  time
    axis:           T
    bounds:         time_bnds

## Add more Global Attributes

In [50]:
from datetime import datetime
current_date = datetime.utcnow().strftime('%Y%m%d')

In [66]:
def add_global_attrs(ds):
    ds.attrs['data_script'] = "Created by program createSolarFileNRLSSI2.ncl"
    ds.attrs['data_script_url'] = "https://svn.code.sf.net/p/codescripts/code/trunk/ncl/solar"
    ds.attrs['cesm_contact'] = "Max Grover, mgrover@ucar.edu"
    ds.attrs['creation_date'] = current_date
    ds.attrs['data_source_url'] = "http://lasp.colorado.edu/lisird/data/nrl2_files"
    ds.attrs['data_source_files'] = infile
    ds.attrs['data_description_url'] = "http://lasp.colorado.edu/lisird/data/nrl2_files"
    ds.attrs['data_reference']    = \
       "Coddington, O., J. Lean, P. Pilewskie, M. Snow, and D. Lindholm (2016), "+\
       "A solar irradiance climate data record, Bull. Amer. Meteor. Soc., "+\
       "doi:10.1175/BAMS-D-14-00265.1."
    ds.attrs['data_summary']    = "Daily SSI calculated using NRL2 solar irradiance model. "
    "Includes spectrally integrated (total) TSI value. " 
    "This dataset contains spectral solar irradiance as a "
    "function of time and wavelength created with the "
    "Naval Research Laboratory model for spectral and "
    "total irradiance (version 2). Spectral solar irradiance "
    "is the wavelength-dependent energy input to the top of "
    "the Earth's atmosphere, at a standard distance "
    "of one Astronomical Unit from the Sun. Also included "
    "is the value of total (spectrally integrated) solar irradiance."
    
    return ds

In [67]:
add_global_attrs(ds)

<xarray.Dataset>
Dimensions:     (time: 50769, wavelength: 3785, bounds: 2)
Coordinates:
  * wavelength  (wavelength) float32 115.5 116.5 117.5 ... 9.992e+04 9.998e+04
  * time        (time) datetime64[ns] 1882-01-01 1882-01-02 ... 2020-12-31
Dimensions without coordinates: bounds
Data variables:
    ssi         (time, wavelength) float32 ...
    band_width  (wavelength) float32 1.0 1.0 1.0 1.0 1.0 ... 50.0 50.0 50.0 50.0
    tsi         (time) float32 1.361e+03 1.361e+03 ... 1.361e+03 1.361e+03
    tsi_unc     (time) float32 0.4545 0.4814 0.4874 ... 0.1955 0.1786 0.1577
    time_bnds   (time, bounds) datetime64[ns] ...
    SSI_UNC     (time, wavelength) float32 ...
Attributes: (12/38)
    Conventions:               CF-1.6, ACDD-1.3
    title:                     Daily SSI calculated using NRL2 solar irradian...
    source:                    NRLSSI2_v02r01
    institution:               Naval Research Laboratory Space Science Divisi...
    standard_name_vocabulary:  CF Standard Name Table v29
    naming_authority:          gov.noaa.ncei
    ...                        ...
    creation_date:             20210909
    data_source_url:           http://lasp.colorado.edu/lisird/data/nrl2_files
    data_source_files:         ../data/ssi_v02r01_daily_s18820101_e20201231_c...
    data_description_url:      http://lasp.colorado.edu/lisird/data/nrl2_files
    data_reference:            Coddington, O., J. Lean, P. Pilewskie, M. Snow...
    data_summary:              Daily SSI calculated using NRL2 solar irradian...

## Breaking down the ncl code

```
load "$CODE_PATH/ncl/lib/common.ncl"

# Setup the paths and get the date
;inPath="/glade/p/acd/mmills/solar/"
inPath="/glade/work/mmills/inputdata/solar/"
creation_date = systemfunc("date +%Y%m%d")
tempFile=inPath+"temp.nc"

# Set the input and output files using the variables specified above
;inFile="ssi_v02r01_daily_s18820101_e20171231_c20180313.nc"
;outFile=inPath+"SolarForcingNRLSSI2_daily_s18820101_e20171231_c"+creation_date+".nc"

inFile="ssi_v02r01-preliminary_daily_s20180101_e20180331_c20180416.nc"
outFile=inPath+"SolarForcingNRLSSI2-preliminary_daily_s20180101_e20180331_c"+creation_date+".nc"

command("rm "+tempFile)
command("rm "+outFile)
command("cp "+inPath+inFile+" "+tempFile)

# Rename variables - including SSI, TSI, TSI_UNC, and band width
command("ncrename -O -v SSI,ssi "+tempFile)
command("ncrename -O -v TSI,tsi "+tempFile)
command("ncrename -O -v TSI_UNC,tsi_unc "+tempFile)
command("ncrename -O -v Wavelength_Band_Width,band_width "+tempFile)

print("opening file")
ncid=addfile(tempFile,"w")

; For a nicer looking netCDF, create a "new line" character.
; This is not necessary.
;
  nl = integertochar(10)  ; newline character


# Add additional metadata
print("writing metadata")
      
ncid@data_script = "Created by program createSolarFileNRLSSI2.ncl"
ncid@data_script_url = "https://svn.code.sf.net/p/codescripts/code/trunk/ncl/solar"
ncid@cesm_contact = "Mike Mills, mmills@ucar.edu"
ncid@creation_date     = systemfunc("date")
ncid@data_source_url = "http://lasp.colorado.edu/lisird/data/nrl2_files"
ncid@data_source_files = inFile
ncid@data_description_url    = "http://lasp.colorado.edu/lisird/data/nrl2_files"
ncid@data_reference    = \
       "Coddington, O., J. Lean, P. Pilewskie, M. Snow, and D. Lindholm (2016), "+\
       "A solar irradiance climate data record, Bull. Amer. Meteor. Soc., "+\
       "doi:10.1175/BAMS-D-14-00265.1."
ncid@data_summary    = \
	"Daily SSI calculated using NRL2 solar irradiance model. "+\ 
        "Includes spectrally integrated (total) TSI value. "+\
	"This dataset contains spectral solar irradiance as a "+\ 
        "function of time and wavelength created with the "+\
        "Naval Research Laboratory model for spectral and "+\ 
        "total irradiance (version 2). Spectral solar irradiance "+\
        "is the wavelength-dependent energy input to the top of "+\ 
        "the Earth's atmosphere, at a standard distance "+\
        "of one Astronomical Unit from the Sun. Also included "+\ 
        "is the value of total (spectrally integrated) solar irradiance."

# Add time variables - take mean over time bounds for time
print("reading time")
;time=time_reassign(ncid,"time")
time=ncid->time
print("reading time_bnds")
time_bnds=ncid->time_bnds
; Fix a bug in the original file with time_bnds
print("correcting time_bnds")
time_bnds(:,0)=(/time/)
time_bnds(:,1)=(/time/)+1
print("correcting time")
time=(/dim_avg(time_bnds)/)
  
print("writing time")
ncid->time=(/time/)
  
print("writing time_bnds")
ncid->time_bnds=(/time_bnds/)


print("calculating date")
date=cd_calendar(time,-2)
date!0="time"
date&time=time
date@units="YYYYMMDD"
printVarSummary(date)

ds=dimsizes(date)

date1=date(0)
date2=date(ds-1)

# Add data title with range of dates
print("Adding data_title")
ncid@data_title = "NRLSSI2 daily solar irradiance for "+date1+"-"+date2
  
print("writing date")
ncid->date=date
print("writing datesec")
datesec=date
datesec=(/toint(3600*24*(time-toint(time)))/)
datesec@units="seconds after midnight UT"
ncid->datesec=datesec

# Scale ssi to different units
print("scaling ssi")
ncid->ssi=1000*ncid->ssi
ncid->ssi@units="mW m-2 nm-1"
ncid->ssi@long_name = "NOAA Climate Data Record of Daily Solar Spectral Irradiance" 

ncid->tsi@ancillary_variables="tsi_unc"

print("closing file "+tempFile)
delete(ncid)


print("deleting ancillary_variables")
command("ncatted -a ancillary_variables,ssi,d,c,, "+tempFile)
;command("ncatted -a ancillary_variables,tsi,d,c,, "+tempFile)

print("File is closed. Removing SSI_UNC")
command("ncks -x -v SSI_UNC "+tempFile+" "+outFile)
;command("rm "+tempFile)
```